<a href="https://colab.research.google.com/github/sagunkayastha/ILab_Tutorials/blob/master/NN_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1

**Use Perceptron Model to classify given data**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
import os

**Data download and import**

In [0]:
!wget https://raw.githubusercontent.com/sagunkayastha/ILab_Tutorials/master/ex2data1.txt
data = np.loadtxt(os.path.join('ex2data1.txt'), delimiter=',')
X, y = data[:, 0:2], data[:, 2]


In [0]:
def plotData(X, y):
    
    pos = y == 1
    neg = y == 0

    plt.plot(X[pos, 0], X[pos, 1], 'kx', lw= 2, ms=8)
    plt.plot(X[neg,0], X[neg,1], 'ko',mfc='y', ms=8, mec='k', mew=1)
    plt.xlabel('Exam 1 score')
    plt.ylabel('Exam 2 score')
    plt.legend(['Admitted', 'Not admitted'])
plotData(X,y)    

**Normalize**

In [0]:
X = X - X.mean()
X = X / X.max()
plotData(X,y)    

**Checking shapes**

In [0]:

y=y.reshape(-1,1)

X,y= X.T,y.T
print('Shape X',X.shape)

print('Shape y',y.shape)

**Algorithm**


![alt text](https://render.githubusercontent.com/render/math?math=z%5E%7B%28i%29%7D%20%3D%20w%5ET%20x%5E%7B%28i%29%7D%20%2B%20b%20%5Ctag%7B1%7D&mode=display)

![alt text](https://render.githubusercontent.com/render/math?math=%5Chat%7By%7D%5E%7B%28i%29%7D%20%3D%20a%5E%7B%28i%29%7D%20%3D%20sigmoid%28z%5E%7B%28i%29%7D%29%5Ctag%7B2%7D&mode=display)

![alt text](https://render.githubusercontent.com/render/math?math=%5Cmathcal%7BL%7D%28a%5E%7B%28i%29%7D%2C%20y%5E%7B%28i%29%7D%29%20%3D%20%20-%20y%5E%7B%28i%29%7D%20%20%5Clog%28a%5E%7B%28i%29%7D%29%20-%20%281-y%5E%7B%28i%29%7D%20%29%20%20%5Clog%281-a%5E%7B%28i%29%7D%29%5Ctag%7B3%7D&mode=display)

**Calculate Loss**

![alt text](https://render.githubusercontent.com/render/math?math=J%20%3D%20%5Cfrac%7B1%7D%7Bm%7D%20%5Csum_%7Bi%3D1%7D%5Em%20%5Cmathcal%7BL%7D%28a%5E%7B%28i%29%7D%2C%20y%5E%7B%28i%29%7D%29%5Ctag%7B6%7D&mode=display)

**Gradient**

![alt text](https://render.githubusercontent.com/render/math?math=%5Cfrac%7B%5Cpartial%20J%7D%7B%5Cpartial%20w%7D%20%3D%20%5Cfrac%7B1%7D%7Bm%7DX%28A-Y%29%5ET%5Ctag%7B7%7D%24%24%24%24%20%5Cfrac%7B%5Cpartial%20J%7D%7B%5Cpartial%20b%7D%20%3D%20%5Cfrac%7B1%7D%7Bm%7D%20%5Csum_%7Bi%3D1%7D%5Em%20%28a%5E%7B%28i%29%7D-y%5E%7B%28i%29%7D%29%5Ctag%7B8%7D&mode=display)

In [0]:
def sigmoid(z):
    
    s = 1 / (1 + np.exp(-z))
    
    return s

## Initialization

In [0]:
def initialize_with_zeros(dim):
    
    w = np.zeros(shape=(dim,1))
    b = 0
    
    return w, b

## Forward Propagation

In [0]:
def propagate(w,b):
  z = np.dot(w.T,X)+b
  A = sigmoid(z)
  
  cost = (- 1 / m) * np.sum(y * np.log(A) + (1 - y) * (np.log(1 - A)))
  
  dw = (1 / m) * np.dot(X, (A - y).T)
  db = (1 / m) * np.sum(A - y)
  cost = np.squeeze(cost)
  
  grads ={'dw':dw,'db':db}
  return grads,cost



In [0]:
def optimize(w,b,learning_rate,num_of_iterations):
  
  for i in range(num_of_iterations):
    grads,cost = propagate(w,b)
    dw= grads['dw']
    db = grads['db']
    w = w - learning_rate * dw  
    b = b - learning_rate * db
    if i % 500 == 0:
      print(f'cost after {i} epochs is {cost}')
      
    params ={'w':w,'b':b}
  return params

learning_rate=0.01

w,b=initialize_with_zeros(2)
m= X.shape[1] #100, number of training examples
params=optimize(w,b,learning_rate,50000)


In [0]:
def predict(params, X):
    
    w=params['w']
    b=params['b']
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        # Convert probabilities a[0,i] to actual predictions p[0,i]
        
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
        
    
    return Y_prediction
predictions = predict(params,X)
print('y = ',y)
print('predictions = ',predictions)

## Final Model

In [0]:
def model(X,y,learning_rate,num_of_iterations):
  #initialization
  w,b = initialize_with_zeros(X.shape[0])
  
  # Forward pass and Gradient descent
  params= optimize(w,b,learning_rate,num_of_iterations)
  
  w = params['w']
  b = params['b']
  
  # Predictions
  predictions =predict(params,X)
  print("accuracy: {} %".format(100 - np.mean(np.abs(predictions - y)) * 100))
  
  
model(X,y,0.01,25000)

# Exercise 2

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

%matplotlib inline

## Data Prep

In [0]:
!wget https://github.com/sagunkayastha/ILab_Tutorials/raw/master/test_catvnoncat.h5
!wget https://github.com/sagunkayastha/ILab_Tutorials/raw/master/train_catvnoncat.h5

In [0]:
# helper function to import data
def load_dataset():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes



**Loading the data (cat/non-cat)**

In [0]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [0]:
# Example
index = 24
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:,index]) + ", it's a '" + classes[np.squeeze(train_set_y[:,index])].decode("utf-8") +  "' picture.")

**Checking Shape**

In [0]:
m_train = train_set_y.shape[1]
m_test = test_set_y.shape[1]
num_px = train_set_x_orig.shape[1]

print ("Number of training examples: m_train = " )
print ("Number of testing examples: m_test = " )
print ("Height/Width of each image: num_px = " )

print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

**Flatten and standardize** 

In [0]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

train_set_x = train_set_x_flatten / 255.
test_set_x = test_set_x_flatten / 255.

![Algorithm](https://github.com/Kulbear/deep-learning-coursera/raw/997fdb2e2db67acd45d29ae418212463a54be06d/Neural%20Networks%20and%20Deep%20Learning/images/LogReg_kiank.png)

## Model

In [0]:
def sigmoid(z):
  pass

In [0]:
def initialize_with_zeros(dim):
  pass

In [0]:
def propagate(w, b, X, Y):
  
  ##Forward pass
  
  ##Backward (Find Gradients) dw, db
  
  ##
  
  
  pass

In [0]:
def optimize(w, b, X, Y, num_iterations, learning_rate):
  pass

**Final Model**

In [0]:
def model(X_train, Y_train, X_test, Y_test, num_iterations, learning_rate=0.5):
  
  pass